<a href="https://colab.research.google.com/github/arigig/ceg-afpm/blob/main/codeT5_base_finueTune_with_DiverseVul_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("arigos/diversevul")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/297442 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/16525 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/16525 [00:00<?, ? examples/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['func', 'target', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message'],
        num_rows: 297442
    })
    test: Dataset({
        features: ['func', 'target', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message'],
        num_rows: 16525
    })
    valid: Dataset({
        features: ['func', 'target', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message'],
        num_rows: 16525
    })
})

In [5]:
# Only taking 10% of whole dataset : train (10%) + test (5%) + validation (5%)
train_test_valid = raw_datasets['train'].train_test_split(test_size=0.1)

In [6]:
train_test_valid

DatasetDict({
    train: Dataset({
        features: ['func', 'target', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message'],
        num_rows: 267697
    })
    test: Dataset({
        features: ['func', 'target', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message'],
        num_rows: 29745
    })
})

In [15]:
# Split the 10% test + valid in half test, half valid
train_data = train_test_valid['test'].train_test_split(test_size=0.5)

In [16]:
train_data

DatasetDict({
    train: Dataset({
        features: ['func', 'target', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message'],
        num_rows: 14872
    })
    test: Dataset({
        features: ['func', 'target', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message'],
        num_rows: 14873
    })
})

In [17]:
valid_test_data = train_data['test'].train_test_split(test_size=0.5)

In [18]:
valid_test_data

DatasetDict({
    train: Dataset({
        features: ['func', 'target', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message'],
        num_rows: 7436
    })
    test: Dataset({
        features: ['func', 'target', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message'],
        num_rows: 7437
    })
})

In [19]:
from datasets import DatasetDict
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_data['train'],
    'test': valid_test_data['test'],
    'valid': valid_test_data['train']})

In [21]:
train_test_valid_dataset['train'].features

{'func': Value(dtype='string', id=None),
 'target': Value(dtype='int64', id=None),
 'cwe': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'project': Value(dtype='string', id=None),
 'commit_id': Value(dtype='string', id=None),
 'hash': Value(dtype='float64', id=None),
 'size': Value(dtype='int64', id=None),
 'message': Value(dtype='string', id=None)}

In [29]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer( example["message"], truncation=True)


tokenized_datasets = train_test_valid_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/14872 [00:00<?, ? examples/s]

Map:   0%|          | 0/7437 [00:00<?, ? examples/s]

Map:   0%|          | 0/7436 [00:00<?, ? examples/s]

In [31]:
tokenized_datasets.rename_column('target','labels')


DatasetDict({
    train: Dataset({
        features: ['func', 'labels', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14872
    })
    test: Dataset({
        features: ['func', 'labels', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7437
    })
    valid: Dataset({
        features: ['func', 'labels', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7436
    })
})

In [32]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ['func', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message']}
[len(x) for x in samples["input_ids"]]

[183, 183, 20, 239, 20, 41, 512, 215]

In [33]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'target': torch.Size([8]),
 'input_ids': torch.Size([8, 512]),
 'token_type_ids': torch.Size([8, 512]),
 'attention_mask': torch.Size([8, 512])}

In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

ImportError: ignored